In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K

n_features = 4
n_runs = 300
seq_length = [180, 360, 540, 720, 900]
labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
labels = labels.drop(columns="Unnamed: 0") 
labels = np.array(labels)

def read_sequences():
    run_list_mix = []
    run_list_ordered = []
    labels_reordered = []
    for index in range(n_runs):
        run_csv = pd.read_csv("Mix_sequences_var_length/run^"+str(index))
        run_csv = run_csv.drop(columns=['Unnamed: 0'])
        run_list_mix.append(run_csv)
    for index, length in enumerate(seq_length):
        run_list_ordered.append([])
        for run_index,run in enumerate(run_list_mix): 
            if len(run) == length:
                run_list_ordered[index].append(run)    
                labels_reordered.append(labels[run_index])
    return run_list_ordered, labels
    
    
    
def stadard_sequences(sequences):
    for index, seqs in enumerate(sequences):
        for i, seq in enumerate(seqs):
            seqs[i] = MinMaxScaler(feature_range=[0,1]).fit_transform(seq)
    return sequences                    


trainX_sequences, labels_ordered = read_sequences()
trainX_sequences = stadard_sequences(trainX_sequences)
for i, leng in enumerate(seq_length):
    trainX_sequences[i] = np.reshape(trainX_sequences[i], (-1, leng, n_features))

# def train_generator():
#     index = 0
#     while True:
#         x_train = trainX_sequences[index]
#         index += 1
#         yield (x_train, x_train) 


TypeError: 'Tensor' object is not callable

In [17]:

from keras.layers import Input, LSTM, RepeatVector, Conv2DTranspose
from keras.models import Model

filters = 50
latent_dim = 20

def Conv1DTranspose(input_tensor, filters, kernel_size,last, strides=1, padding='same'):
        if last:
            activation = 'linear'
        else:
            activation = 'relu'
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation=activation)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


def repeat(x):
    stepMatrix = K.ones_like(x[0][:,:,:1]) #matrix with ones, shaped as (batch, steps, 1)
    latentMatrix = K.expand_dims(x[1],axis=1) #latent vars, shaped as (batch, 1, latent_dim)

    return K.batch_dot(stepMatrix, latentMatrix)


inputs = Input(shape=(None, n_features))
x = inputs
for i in range(2):
    x = Conv1D(filters=filters, kernel_size=2, activation='relu', padding='same')(x)
   
   
encoded = LSTM(20)(x)
#encoded = K.batch_flatten(x)
encoded = Dense(latent_dim)(encoded)
x = Lambda(repeat)([inputs,encoded])

for i in range(2):
    x = Conv1DTranspose(input_tensor=x, filters=filters, kernel_size=2, padding='same', last=False)
   
#decoded = LSTM(n_features, return_sequences=True)(x)

output = Dense(n_features)(x)

sequence_autoencoder = Model(inputs, output)
encoder = Model(inputs, encoded)


sequence_autoencoder.summary()
sequence_autoencoder.compile(optimizer='adam', loss='mse')

TypeError: 'Tensor' object is not callable

In [3]:
def train():
    for batch in trainX_sequences:
        if len(batch) != 0:
            sequence_autoencoder.fit(batch, batch, epochs=50, verbose=1)


train()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/300



 32/300 [==>...........................] - ETA: 18s - loss: 0.3559

 64/300 [=====>........................] - ETA: 8s - loss: 0.3453 

 96/300 [========>.....................] - ETA: 5s - loss: 0.3367

...

192/300 [==================>...........] - ETA: 1s - loss: 0.1220

224/300 [=====================>........] - ETA: 1s - loss: 0.1220

256/300 [========================>.....] - ETA: 0s - loss: 0.1220

288/300 [===========================>..] - ETA: 0s - loss: 0.1220

300/300 [==============================] - 5s 16ms/step - loss: 0.1220


In [5]:
from sklearn.decomposition import PCA

encodings = []
for i,seqs in enumerate(trainX_sequences): 
    if len(seqs) != 0:
        encodings.append(encoder.predict(seqs))
    
encodings_squeezed = []
print(len(encodings), encodings[0].shape)

1 (300, 20)


In [6]:

for i in encodings:
    for j in i:
        encodings_squeezed.append(j)
encs = np.array(encodings_squeezed)
print(encs.shape)


(300, 20)


In [7]:
def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    return arg

    
masks = [return_mask(num, np.array(labels_ordered))[:, 0] for num in range(0, 9)]


(300, 3)
[0.54483753 0.28783548 0.06855268]


In [8]:

from mpl_toolkits.mplot3d import Axes3D  


def plot_pca(title, i): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])
        #break
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask],
                    alpha=0.5)
        #break
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    

scaler = StandardScaler()
enc_input = scaler.fit_transform(encs) 
pca = PCA(3)
principalComponents = pca.fit_transform(enc_input)
print(principalComponents.shape)
print(pca.explained_variance_ratio_)
plot_pca('Sequences', 0)

principalComponents = encs
plot_pca('Sequences_Not_Pca', 0)



(300, 3)
[0.54483753 0.28783548 0.06855268]
